In [1]:
import decoupler as dc
import pandas as pd

In [2]:
help(dc.translate_net)

Help on function translate_net in module decoupler.omnip:

translate_net(net: 'pd.DataFrame', columns: 'str | Iterable[str]' = ('source', 'target', 'genesymbol'), source_organism: 'str | int' = 'human', target_organism: 'str | int' = 'mouse', id_type: 'str | tuple[str, str]' = 'genesymbol', unique_by: 'Iterable[str] | None' = ('source', 'target'), **kwargs: 'dict[str, str]') -> 'pd.DataFrame'
    Translate networks between species by orthology.
    
    This function downloads orthology databases from omnipath and converts
    genes between species. The first time you run this function will take a
    while (~15 minutes) but then it stores all the information in cache for
    quick reusability.
    
    In case you need to reset the cache, you can do it by doing:
    ``rm -r ~/.pypath/cache/``.
    
    With its default parameters, this function translates almost any network
    or annotation data frame acquired by the functions in this module from
    human to mouse. For the PROGENy r

In [3]:
df = pd.read_csv('../data/gene_names.csv')
#df.info()
hs = df[df["spec"] == "Hs"]
mm = df[df["spec"] == "Mm"]
rn = df[df["spec"] == "Rn"]

In [ ]:
hs_to_mm = dc.translate_net(hs, columns="gene", source_organism="human", target_organism="mouse")
hs_to_mm

KeyboardInterrupt: 

In [ ]:
rn_to_mm = dc.translate_net(rn, columns="gene", source_organism="rat", target_organism="mouse")

In [ ]:
print(df)